In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 19 04:45:47 2025

@author: ngoni97
"""
%matplotlib inline
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# loading machine learning modules
import sklearn
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

from dataset_collector_saver_class import LoadDataset


In [ ]:
# test directory for mathematics
physics_path = '/home/ngoni97/Documents/PHYSICS'

data = LoadDataset(physics_path, 
                      tar_names=['ADVANCED'], 
                      Type='documents', 
                      rmChar=True, _dict=True, 
                     folder_name=True)
physics_dataset = data.returnDataset()
Data = np.array(physics_dataset[0])
Data


array(['the quantum theory of fields volume 1  pdfdrive ',
       'solid state physics  lecturenotes2012',
       'quantum field theory and the standard model  pdfdrive ',
       'pisello gravitationelectromagnetismquantisedcharge',
       'i v savelyev fundametals of theoretical physics vol 1',
       'hydraulic machines textbook',
       'compendium of theoretical physics,springer2005,529p,0387257993',
       'introduction to theoretical particle physics',
       'statistical physics  pdfdrive ',
       '50 physics ideas you really need to know  joanne baker',
       'quantum mechanics the theoretical minimum leonard susskind and art friedman',
       '1 mod 1 atomic nuclear physics', 'han ohanian 2',
       'slaterfrank introductiontotheoreticalphysics',
       'thermodynamics  statistical mechanics by stowe', 'han ohanian 1',
       'statistical field theory  an introduction to exactly solved models in statistical physics   pdfdrive ',
       'introduction to solid state physics  p

In [3]:
"""
Examples
--------
>>> from sklearn.feature_extraction.text import TfidfVectorizer
>>> corpus = [
...     'This is the first document.',
...     'This document is the second document.',
...     'And this is the third one.',
...     'Is this the first document?',
... ]
>>> vectorizer = TfidfVectorizer()
>>> X = vectorizer.fit_transform(corpus)
>>> vectorizer.get_feature_names_out()
array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], ...)
>>> print(X.shape)
(4, 9)
"""

vectoriser = TfidfVectorizer()
X = vectoriser.fit_transform(Data)
vectoriser.get_feature_names_out()


array(['000', '0387257993', '05', '1000', '10th', '12', '13th', '1612',
       '1962t656s', '1978', '1989', '1st', '200', '2004', '2008', '2009',
       '200pp', '2010', '2015', '2016', '2017', '20th', '250', '2500',
       '28', '29', '2nd', '3ed', '3rd', '4369', '4th', '50', '529p',
       '5th', '6th', '8th', 'advanced', 'advances', 'age', 'algorithms',
       'alternative', 'altland', 'an', 'analysis', 'analytical', 'and',
       'andreas', 'applications', 'applied', 'approach', 'art', 'artech',
       'asian', 'atkins', 'atomic', 'atoms', 'automaton', 'babbar',
       'bahaa', 'baker', 'bang', 'baryons', 'basic', 'basics',
       'beginners', 'beiser', 'bernard', 'between', 'big', 'board',
       'body', 'bohr', 'book', 'boson', 'brevik', 'bridge',
       'briefwechsel', 'by', 'cambridge', 'cbse', 'cellular', 'century',
       'ch15', 'chaikin', 'chapters', 'charles', 'chemistry', 'class',
       'classical', 'classicalmechanics', 'collected', 'college',
       'collegephysics', '

In [4]:
labels = ['ADVANCED']*len(Data)

df = pd.DataFrame({'Document': Data, 'Category':labels})
display(df[:30])


,Document,Category
0,the quantum theory of fields volume 1 pdfdrive,ADVANCED
1,solid state physics lecturenotes2012,ADVANCED
2,quantum field theory and the standard model p...,ADVANCED
3,pisello gravitationelectromagnetismquantisedch...,ADVANCED
4,i v savelyev fundametals of theoretical physic...,ADVANCED
5,hydraulic machines textbook,ADVANCED
6,"compendium of theoretical physics,springer2005...",ADVANCED
7,introduction to theoretical particle physics,ADVANCED
8,statistical physics pdfdrive,ADVANCED
9,50 physics ideas you really need to know joan...,ADVANCED


In [5]:
stop_words = nltk.corpus.stopwords.words('english')

wpt = nltk.WordPunctTokenizer()
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

Norm = normalize_corpus(Data)
Norm


array(['quantum theory fields volume 1 pdfdrive',
       'solid state physics lecturenotes2012',
       'quantum field theory standard model pdfdrive',
       'pisello gravitationelectromagnetismquantisedcharge',
       'v savelyev fundametals theoretical physics vol 1',
       'hydraulic machines textbook',
       'compendium theoretical physicsspringer2005529p , 0387257993',
       'introduction theoretical particle physics',
       'statistical physics pdfdrive',
       '50 physics ideas really need know joanne baker',
       'quantum mechanics theoretical minimum leonard susskind art friedman',
       '1 mod 1 atomic nuclear physics', 'han ohanian 2',
       'slaterfrank introductiontotheoreticalphysics',
       'thermodynamics statistical mechanics stowe', 'han ohanian 1',
       'statistical field theory introduction exactly solved models statistical physics pdfdrive',
       'introduction solid state physics pdfdrive', 'wilsonchap',
       'lecture notes mathematical methods the

In [6]:
cv = CountVectorizer(min_df=0., max_df=1.)

cv_matrix = cv.fit_transform(Norm)
cv_matrix = cv_matrix.toarray()
print(cv_matrix)

columns = cv.get_feature_names_out()
Df = pd.DataFrame(cv_matrix, columns=columns)
display(Df)
vocab = cv.get_feature_names_out()
vocab


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


,0387257993,05,1000,10th,12,13th,1612,1962t656s,1978,1989,...,wolfgang,wordpress,worked,world,yan,young,zemanskyrichard,zemanskys,zhang,zheng
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


array(['0387257993', '05', '1000', '10th', '12', '13th', '1612',
       '1962t656s', '1978', '1989', '1st', '200', '2004', '2008', '2009',
       '200pp', '2010', '2015', '2016', '2017', '20th', '250', '2500',
       '28', '29', '2nd', '3000', '3ed', '3rd', '4369', '4th', '50',
       '5th', '6th', '8th', 'advanced', 'advances', 'age', 'algorithms',
       'alternative', 'altland', 'analysis', 'analytical', 'andreas',
       'applications', 'applied', 'approach', 'art', 'artech', 'asian',
       'atkins', 'atomic', 'atoms', 'automaton', 'babbar', 'bahaa',
       'baker', 'bang', 'baryons', 'basic', 'basics', 'beginners',
       'beiser', 'bernard', 'big', 'board', 'body', 'bohr', 'book',
       'boson', 'brevik', 'bridge', 'briefwechsel', 'cambridge', 'cbse',
       'cellular', 'century', 'ch15', 'chaikin', 'chapters', 'charles',
       'chemistry', 'class', 'classical', 'classicalmechanics',
       'collected', 'college', 'collegephysics', 'collins', 'colombo',
       'commentary', 'c

In [7]:
tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(Norm)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names_out()
DF = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)
display(DF)
tv_matrix


,0387257993,05,1000,10th,12,13th,1612,1962t656s,1978,1989,...,wolfgang,wordpress,worked,world,yan,young,zemanskyrichard,zemanskys,zhang,zheng
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
similarity_matrix = cosine_similarity(tv_matrix)
my_df = pd.DataFrame(similarity_matrix)
display(my_df)


,0,1,2,3,4,5,6,7,8,9,...,301,302,303,304,305,306,307,308,309,310
0,1.000000,0.000000,0.264493,0.0,0.000000,0.0,0.000000,0.000000,0.091011,0.000000,...,0.000000,0.097675,0.0,0.044829,0.047548,0.000000,0.0,0.0,0.000000,0.068779
1,0.000000,1.000000,0.000000,0.0,0.039563,0.0,0.000000,0.053170,0.080064,0.024700,...,0.342482,0.000000,0.0,0.000000,0.041829,0.067150,0.0,0.0,0.044308,0.000000
2,0.264493,0.000000,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.075896,0.000000,...,0.000000,0.081452,0.0,0.037384,0.039651,0.000000,0.0,0.0,0.000000,0.057356
3,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,0.000000,0.039563,0.000000,0.0,1.000000,0.0,0.111708,0.208164,0.064933,0.020032,...,0.000000,0.000000,0.0,0.000000,0.033924,0.054460,0.0,0.0,0.035935,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0.000000,0.067150,0.000000,0.0,0.054460,0.0,0.000000,0.073191,0.110213,0.034001,...,0.000000,0.000000,0.0,0.000000,0.057580,1.000000,0.0,0.0,0.060993,0.000000
307,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000
308,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000
309,0.000000,0.044308,0.000000,0.0,0.035935,0.0,0.000000,0.048294,0.072722,0.022435,...,0.000000,0.000000,0.0,0.000000,0.206937,0.060993,0.0,0.0,1.000000,0.000000


Another technique is Latent Dirichlet Allocation (LDA), which uses a generative probabilistic model
where each document consists of a combination of several topics and each term or word can be assigned to
a specific topic. 

In [9]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=8, max_iter=100, random_state=42)
dt_matrix = lda.fit_transform(tv_matrix)
columns = [i for i in range(8)]
features = pd.DataFrame(dt_matrix, columns=columns)
display(features)


,0,1,2,3,4,5,6,7
0,0.040059,0.040071,0.040036,0.719534,0.040049,0.040052,0.040036,0.040162
1,0.321672,0.043742,0.043736,0.043740,0.415892,0.043743,0.043735,0.043741
2,0.037758,0.037760,0.037750,0.037762,0.215554,0.037755,0.037750,0.557912
3,0.051777,0.051777,0.051777,0.051777,0.051777,0.051777,0.637563,0.051777
4,0.719047,0.040141,0.040119,0.040135,0.040133,0.040184,0.040118,0.040123
...,...,...,...,...,...,...,...,...
306,0.055407,0.055404,0.055395,0.055401,0.612192,0.055406,0.055392,0.055403
307,0.046235,0.046237,0.046237,0.046244,0.046236,0.046237,0.676316,0.046257
308,0.038770,0.038760,0.038760,0.038763,0.038760,0.728625,0.038760,0.038803
309,0.043618,0.043700,0.694170,0.043628,0.043756,0.043876,0.043572,0.043678


Limitations of K-means
- K-means clustering needs the number of clusters to be specified.
- K-means has problems when clusters are of differing sized, densities, and non-globular shapes.
- Presence of outlier can skew the results.

In [10]:
# topic model one
km = KMeans()
km.fit_transform(similarity_matrix) # after cosine_similarity modelling
cluster_labels = km.labels_
my_cluster_labels = []
for clt in cluster_labels:
    if clt == 0:
        my_cluster_labels.append(clt)
new_cluster = np.array(my_cluster_labels)
print('topic_model_one:\n',cluster_labels)
# topic model two
KM = KMeans()
KM.fit_transform(features) # after LatentDirichletAllocation modelling
Cluster_labels = KM.labels_
print('\n\ntopic_model_two:\n',Cluster_labels)
print('\n\nnew_cluster:\n', new_cluster)


topic_model_one:
 [3 2 3 0 4 0 0 5 6 0 1 4 0 0 6 0 6 2 0 4 4 0 4 4 5 3 1 4 5 4 0 0 1 3 3 0 4
 2 4 1 4 7 3 4 0 0 6 2 1 3 6 0 0 0 2 5 0 4 4 0 4 3 0 0 1 2 5 6 6 4 2 1 7 3
 0 4 0 2 5 3 0 5 0 4 4 2 0 0 0 4 5 1 1 4 0 6 7 4 0 4 0 6 2 4 4 1 0 6 0 0 4
 4 4 0 4 3 2 4 3 0 5 1 0 0 6 0 3 4 3 4 1 1 0 1 1 0 1 4 0 1 0 4 4 2 2 4 0 0
 2 0 1 7 0 4 4 0 0 3 0 0 1 1 0 0 2 4 1 0 4 4 1 0 4 3 5 4 4 0 4 1 3 0 0 4 5
 4 0 6 4 7 0 0 4 7 0 1 0 1 0 1 5 7 1 0 0 1 4 1 4 2 2 1 0 0 0 0 4 0 7 0 4 0
 4 0 6 4 1 4 5 7 4 1 5 4 4 4 0 4 0 0 3 1 4 0 4 4 0 0 4 4 1 0 2 4 0 0 0 7 0
 0 4 0 0 4 0 0 1 0 0 0 4 2 0 0 1 1 6 4 4 4 0 6 0 4 4 2 4 3 1 0 1 5 4 0 5 0
 4 6 0 4 4 2 1 0 7 4 4 0 0 4 1]


topic_model_two:
 [1 2 0 4 7 1 6 6 3 7 1 2 5 0 3 5 3 2 7 7 6 5 0 2 1 1 1 7 3 7 4 3 0 7 0 2 2
 2 2 0 7 3 1 3 1 5 3 2 0 0 2 7 6 3 2 6 1 6 0 1 1 0 0 2 0 2 0 3 0 3 2 0 0 1
 7 4 5 2 0 0 3 0 7 7 0 2 5 7 2 1 6 7 1 6 5 0 3 1 3 2 0 3 7 7 7 3 1 0 2 0 2
 5 7 5 6 0 7 2 0 2 3 4 1 6 0 1 5 2 0 3 1 6 6 0 0 4 0 7 4 6 0 3 7 2 2 4 4 0
 2 4 0 3 0 0 2 5 0 0 1 3 0 0 2

In [11]:
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
df_1 = pd.concat([df, cluster_labels], axis=1)
df_1[:30]


,Document,Category,ClusterLabel
0,the quantum theory of fields volume 1 pdfdrive,ADVANCED,3
1,solid state physics lecturenotes2012,ADVANCED,2
2,quantum field theory and the standard model p...,ADVANCED,3
3,pisello gravitationelectromagnetismquantisedch...,ADVANCED,0
4,i v savelyev fundametals of theoretical physic...,ADVANCED,4
5,hydraulic machines textbook,ADVANCED,0
6,"compendium of theoretical physics,springer2005...",ADVANCED,0
7,introduction to theoretical particle physics,ADVANCED,5
8,statistical physics pdfdrive,ADVANCED,6
9,50 physics ideas you really need to know joan...,ADVANCED,0



#df["col"][row_indexer] = value

#Use `df.loc[row_indexer, "col"] = values` instead,
#to perform the assignment in a single step and ensure this keeps updating the original `df`.
 
for item in range(len(df_1['ClusterLabel'])):
    if df_1.loc[item,'ClusterLabel'] == 0:
        df_1.loc[item,'ClusterLabel'] = 9999
df_1[:29]

In [12]:
Cluster_labels = pd.DataFrame(Cluster_labels, columns=['ClusterLabel'])
df_2 = pd.concat([df, Cluster_labels], axis=1)
df_2[:30]


,Document,Category,ClusterLabel
0,the quantum theory of fields volume 1 pdfdrive,ADVANCED,1
1,solid state physics lecturenotes2012,ADVANCED,2
2,quantum field theory and the standard model p...,ADVANCED,0
3,pisello gravitationelectromagnetismquantisedch...,ADVANCED,4
4,i v savelyev fundametals of theoretical physic...,ADVANCED,7
5,hydraulic machines textbook,ADVANCED,1
6,"compendium of theoretical physics,springer2005...",ADVANCED,6
7,introduction to theoretical particle physics,ADVANCED,6
8,statistical physics pdfdrive,ADVANCED,3
9,50 physics ideas you really need to know joan...,ADVANCED,7


It appears that after running/using two topic-model based features, the clustering categories changed noticeably

In [13]:
topic_term_matrix = lda.components_
for topic_weights in topic_term_matrix:
    topic = [(token, weight) for token, weight in zip(vocab, topic_weights)]
    topic = sorted(topic, key=lambda x: -x[1])
    topic = [item for item in topic if item[1] > 1.0]
    print(topic)
    print(len(topic))


[('physics', 4.742837378925728), ('matter', 3.7519516004324576), ('condensed', 3.452892363976407), ('compress', 2.597144082797946), ('vol', 2.4205105526373063), ('methods', 2.2231597972335186), ('mathematical', 2.193031265854111), ('theory', 2.139216371221282), ('introduction', 2.0150046825887067), ('theoretical', 1.9910060047398093), ('pdfdrive', 1.9066886695547542), ('fundamentals', 1.8207381517988763), ('elasticity', 1.5501343949111126), ('treatise', 1.5384471847242203), ('8th', 1.5200380462316414), ('practical', 1.4491073499507507), ('atomic', 1.4209421274121312), ('edition', 1.380108338543721), ('fundametals', 1.247545038888498), ('savelyev', 1.247545038888498), ('classicalmechanics', 1.124999126624218), ('wilsonchap', 1.124999126624212), ('renorm', 1.124999126624161), ('symon', 1.0371186687516118), ('olympiad', 1.0175635639044454), ('electricity', 1.0043797899607245), ('magnetism', 1.0043797899607245), ('maxwell', 1.0043797899607245)]
28
[('statistical', 5.785396183294678), ('eng

# Word Embeddings

In [14]:
from gensim.models import word2vec

wpt = nltk.WordPunctTokenizer()


In [15]:
tokenized_my_data = [wpt.tokenize(document) for document in Norm]

# set values for various parameters
feature_size = 10 # word vector dimensionality
window_context = 10 # context window size
min_word_count = 1 # minimum word count
sample = 1e-3 # downsample setting for frequent words

w2v_model = word2vec.Word2Vec(tokenized_my_data, vector_size=feature_size,
                              window=window_context, min_count=min_word_count,
                              sample=sample)


In [16]:
w2v_model.wv['physics']


array([ 0.01002514, -0.0043416 ,  0.06961443,  0.09342295, -0.08602922,
       -0.0813191 ,  0.08532396,  0.10856104, -0.07910226, -0.04486759],
      dtype=float32)